In [1]:
from sklearn.preprocessing import LabelEncoder

In [2]:
import pandas as pd

#读取训练集，此时训练集手工已去掉6个重复样本
train = pd.read_excel('Dataset/训练集.xlsx', index_col='ID')

In [3]:
testa = pd.read_excel('Dataset/测试集A.xlsx', index_col='ID')

In [4]:
data, data_a = train, testa

In [5]:
print(data.info(), data_a.info())

<class 'pandas.core.frame.DataFrame'>
Index: 794 entries, NH0995 to NH1942
Columns: 5953 entries, TOOL to Value
dtypes: float64(5183), int64(759), object(11)
memory usage: 36.1+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, NH1943 to NH0463
Columns: 5952 entries, TOOL to 750X1452
dtypes: float64(5289), int64(652), object(11)
memory usage: 13.6+ MB
None None


In [6]:
data = data.drop(['NH1835'], axis=0)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 793 entries, NH0995 to NH1942
Columns: 5953 entries, TOOL to Value
dtypes: float64(5183), int64(759), object(11)
memory usage: 36.0+ MB
None


In [7]:
# 将训练集的文本数据转换为数值型数据
le = LabelEncoder()
data["TOOL"] = le.fit_transform(data["TOOL"])
data["Tool"] = le.fit_transform(data["Tool"])
data["TOOL_ID"] = le.fit_transform(data["TOOL_ID"])
data["Tool (#1)"] = le.fit_transform(data["Tool (#1)"])
data["TOOL (#1)"] = le.fit_transform(data["TOOL (#1)"])
data["TOOL (#2)"] = le.fit_transform(data["TOOL (#2)"])
data["Tool (#2)"] = le.fit_transform(data["Tool (#2)"])
data["Tool (#3)"] = le.fit_transform(data["Tool (#3)"])
data["Tool (#4)"] = le.fit_transform(data["Tool (#4)"])
data["Tool (#5)"] = le.fit_transform(data["Tool (#5)"])
data["TOOL (#3)"] = le.fit_transform(data["TOOL (#3)"])
data["OPERATION_ID"] = le.fit_transform(data["OPERATION_ID"])

In [8]:
# 将测试集a的文本数据转换为数值型数据
le = LabelEncoder()
data_a["TOOL"] = le.fit_transform(data_a["TOOL"])
data_a["Tool"] = le.fit_transform(data_a["Tool"])
data_a["TOOL_ID"] = le.fit_transform(data_a["TOOL_ID"])
data_a["Tool (#1)"] = le.fit_transform(data_a["Tool (#1)"])
data_a["TOOL (#1)"] = le.fit_transform(data_a["TOOL (#1)"])
data_a["TOOL (#2)"] = le.fit_transform(data_a["TOOL (#2)"])
data_a["Tool (#2)"] = le.fit_transform(data_a["Tool (#2)"])
data_a["Tool (#3)"] = le.fit_transform(data_a["Tool (#3)"])
data_a["Tool (#4)"] = le.fit_transform(data_a["Tool (#4)"])
data_a["Tool (#5)"] = le.fit_transform(data_a["Tool (#5)"])
data_a["TOOL (#3)"] = le.fit_transform(data_a["TOOL (#3)"])
data_a["OPERATION_ID"] = le.fit_transform(data_a["OPERATION_ID"])

In [9]:
# 测试集a中，缺失比例大于10%的特征，用0填充，其余缺失值用均值填充
for i in data_a.columns:
    average = data_a[i].mean()
    if data_a[i].count() / len(data_a) < 0.1:
        data_a[i] = data_a[i].fillna(0)
    else:
        data_a[i] = data_a[i].fillna(average)

print(data_a.info())

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, NH1943 to NH0463
Columns: 5952 entries, TOOL to 750X1452
dtypes: float64(5289), int32(11), int64(652)
memory usage: 13.6+ MB
None


In [10]:
#删除训练集和测试集A差距很大的列
diff_list = []
for column in data.columns:
    cnt = 0
    if column != "Value":
        minm = data[column].min()
        maxm = data[column].max()
        testdata_list = data_a[column].tolist()
        for listdata in testdata_list:
            if listdata > maxm or listdata < minm:
                cnt = cnt + 1
        if cnt >= 50:
            diff_list.append(column)
print(diff_list)
data = data.drop(diff_list, axis=1)
print(data.info())

['TOOL', '210X95', '210X204', '210X205', '210X213', '210X215', '220X33', '220X67', '220X71', '220X75', '220X79', '220X83', '220X87', '220X91', '220X95', '220X150', '220X151', '220X153', '220X154', '220X155', '220X156', '220X157', '220X159', '220X160', '220X161', '220X163', '220X170', '220X171', '220X172', '220X173', '220X176', '220X180', '220X181', '220X182', '220X183', '220X184', '220X185', '220X187', '220X188', '220X189', '220X194', '220X195', '220X196', '220X197', '220X198', '220X200', '220X201', '220X202', '220X204', '220X206', '220X207', '220X208', '220X212', '220X213', '220X216', '220X217', '220X218', '220X219', '300X2', '300X3', '300X4', '300X6', '300X7', '300X9', '300X10', '300X13', '300X14', '300X20', '310X56', '310X60', '310X64', '310X68', '310X72', '310X76', '310X80', '310X84', '310X207', '311X5', '311X6', '311X7', '311X20', '311X22', '311X23', '311X25', '311X26', '311X28', '311X54', '311X55', '311X56', '311X58', '311X59', '311X60', '311X77', '311X78', '311X79', '311X162', '

In [11]:
# 方差为0的特征，删除
del_var = []
for i in data.columns:
    if data[i].var() == 0:
        del_var.append(i)
data = data.drop(del_var, axis=1)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 793 entries, NH0995 to NH1942
Columns: 4861 entries, 210X1 to Value
dtypes: float64(4558), int32(10), int64(293)
memory usage: 29.4+ MB
None


In [12]:
# 训练集中，缺失比例大于10%的特征，删除，其余缺失值用均值填充
for i in data.columns:
    average = data[i].mean()
    if data[i].count() / len(data) < 0.1:
        data = data.drop(i, axis=1)
    else:
        data[i] = data[i].fillna(average)

print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 793 entries, NH0995 to NH1942
Columns: 4634 entries, 210X1 to Value
dtypes: float64(4331), int32(10), int64(293)
memory usage: 28.0+ MB
None


In [13]:
# 删除日期列
for i in data.columns:
    max = data[i].max()
    if max > 1e10 and (str(max).find('2016') != -1 or str(max).find('2017') != -1):
        data = data.drop(i, axis=1)

print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 793 entries, NH0995 to NH1942
Columns: 4633 entries, 210X1 to Value
dtypes: float64(4330), int32(10), int64(293)
memory usage: 28.0+ MB
None


In [14]:
#特征间相关性（线性化）过高的特征，只保留一条以显示信息，其余删除；
cor = []
tmp = []
matrix = data.corr()

for i in matrix.columns:
    if i in matrix.columns:
        index1 = matrix[matrix[i] >= 0.96].index
        index2 = matrix[matrix[i] <= -0.96].index
        index = list(index1) + list(index2)
        for j in index:
            if j != i:
                tmp.append(j)
                cor.append(j)
        if len(tmp) != 0:
            matrix = matrix.drop(tmp, axis=0)
            matrix = matrix.drop(tmp, axis=1)
            tmp = []
print(len(cor))
data = data.drop(cor, axis=1)
print(data.info())

3277
<class 'pandas.core.frame.DataFrame'>
Index: 793 entries, NH0995 to NH1942
Columns: 1356 entries, 210X1 to Value
dtypes: float64(1199), int32(10), int64(147)
memory usage: 8.2+ MB
None


In [15]:
data.to_excel('Dataset/train.xlsx')

In [16]:
#输出测试集a为excel文件
data_a.to_excel('Dataset/testa.xlsx')